In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xml.etree.ElementTree
import os,glob,sys
import pandas as pd
from sklearn.decomposition import PCA
import pylab as plt
import seaborn as sns
import numpy as np

sys.path.append("../..")
from mzmine import pick_peaks, align, match_aligned_to_original, load_aligned_peaks

In [3]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/17_20_PhD/19_20_PhD_Metabolomics/'

osp = MAC

### Load text files with aligned data


In [116]:
with_mod = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/samples_alignment/limma_independent_toptable_features.csv', sep = ',')
feat_des = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/feature_description.csv', sep = ',')

without_mod = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/topfeatures50_filled.txt', sep = '\t')

In [117]:
zik = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/zik/limma_independent_toptable_features.csv', 
                  sep = ',')
mal = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/mal_changed_rt/limma_independent_toptable_features.csv', 
                  sep = ',')
vl = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/vl_changed_rt/limma_independent_toptable_features.csv', 
                 sep = ',')

In [135]:
output_dir = osp+'/pymz/mzmine/peak_picked_files/'

In [84]:
with_mod

,row m/z,row retention time,adj-p,t-val,custom_id
0,832.58,3.16,0.00,-8.69,3022
1,213.02,11.90,0.00,-8.57,1517
2,787.67,3.99,0.00,-8.22,2947
3,203.05,12.05,0.00,7.89,1401
4,101.07,11.93,0.00,-7.77,101
...,...,...,...,...,...
599,170.09,10.55,0.98,-0.03,10779
600,268.14,9.22,0.98,0.03,2057
601,343.30,4.11,0.99,0.02,2476
602,202.14,4.38,0.99,-0.01,1393


In [93]:
def check_p_val_for_peak(dataset, mz, rt):
    for _, row in dataset.iterrows():
        dmz = row['row m/z']
        drt = row['row retention time']
        dpval = row['adj-p']
        
        if in_range(mz, dmz, 0.01):
            if in_range(rt, drt, 0.5):
                return dpval

In [94]:
def add_individual_pval_to_combined(combined, individual, colname):
    for index,row in combined.iterrows():
        combined_mz = row['row m/z']
        combined_rt = row['row retention time']
        combined_pval = row['adj-p']

        combined.at[index, colname] = check_p_val_for_peak(individual, combined_mz, combined_rt)

In [95]:
def in_range(x, y, tolerance):
    if x >= y - tolerance and x <= y+tolerance:
        return True
    

In [118]:
with_mod['vl'] = None
add_individual_pval_to_combined(with_mod, vl, 'vl')

In [119]:
with_mod['mal'] = None
add_individual_pval_to_combined(with_mod, mal, 'mal')

In [120]:
with_mod['zik'] = None
add_individual_pval_to_combined(with_mod, zik, 'zik')

In [23]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [99]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(with_mod)

     row m/z  row retention time  adj-p  t-val  custom_id   vl  mal  zik
0     832.58                3.16   0.00  -8.69       3022 0.00 0.45 0.00
1     213.02               11.90   0.00  -8.57       1517 0.00 0.25 0.55
2     787.67                3.99   0.00  -8.22       2947 0.00 0.00 0.00
3     203.05               12.05   0.00   7.89       1401 0.00 0.50 0.51
4     101.07               11.93   0.00  -7.77        101 0.00 0.70 0.70
5     468.31                3.97   0.00  -7.69       2664 0.00 0.01 0.61
6     809.59                3.19   0.00  -7.61       2987 0.00 0.69 0.86
7     280.24                3.08   0.00   7.58       2175 0.00 0.96 0.33
8     808.58                3.17   0.00  -7.30       2985 0.00 0.64 0.82
9     766.57                3.19   0.00  -7.22       2912 0.00 0.77 0.79
10    380.26                6.71   0.00  -7.22       2549 0.00 0.20 0.05
11    147.08               11.91   0.00  -7.21        613 0.00 0.01 0.00
12    149.08               11.91   0.00  -7.19     

In [121]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(with_mod)

     row m/z  row retention time  adj-p  t-val  custom_id   vl  mal  zik
0     148.07               11.92   0.00  -7.23        633 0.00 1.00 0.99
1     147.08               11.91   0.00  -7.05        613 0.00 0.83 0.60
2     905.59                3.00   0.00  -7.05       3109 0.00 0.90 0.76
3     810.60                3.17   0.00  -6.77       2989 0.00 0.87 0.99
4     126.02               12.21   0.00  -6.77        324 0.01 1.00 0.60
5     782.57                3.20   0.00  -6.67       2936 0.00 0.90 0.99
6     812.61                3.17   0.00  -6.55       2992 0.00 0.89 0.99
7     784.58                3.20   0.00  -6.53       2938 0.00 1.00 0.92
8     835.60                3.16   0.00  -6.52       3026 0.00 0.83 0.81
9     171.08               11.89   0.00  -6.47        951 0.00 1.00 0.99
10    169.06               11.90   0.00  -6.45        926 0.00 0.99 0.99
11    785.59                3.20   0.00  -6.44       2939 0.00 1.00 0.89
12    101.07               11.93   0.00  -6.42     

In [132]:
cids = []
for _,rows in with_mod.iterrows():
    vl = rows['vl']
    mal = rows['mal']
    zik = rows['zik']
    pval = rows['adj-p']
    cid = rows['custom_id']
    
    if pval <= 0.05:

            if vl > 0.05 and mal > 0.05 and zik > 0.05:
                cids.append(cid)
            
        

In [133]:
len(cids)

64

In [137]:
import pickle
with open(output_dir + 'metabolites_insig_individ.txt', 'wb') as handle:
    pickle.dump(cids, handle)

In [58]:
cids.to_csv(output_dir+'list_of_ids_insignificant', header=True, index=None, sep=',', mode='w')

AttributeError: 'list' object has no attribute 'to_csv'

In [163]:
MNET_PATH = '/Users/anamaria/git/molnet/code/'
sys.path.append(MNET_PATH)
with open(os.path.join(osp,'pymz/mzmine/HMDBannodat.csv'), 'rb') as file:
    ann = pickle.load(file)

In [166]:
with_mod.loc[324]

row m/z              236.07
row retention time     4.67
adj-p                  0.31
t-val                 -1.40
custom_id              1798
vl                     0.03
mal                    0.76
zik                    0.60
Name: 324, dtype: object

In [164]:
ann

,row m/z,row retention time,adj p-val,t-score,custom_id,compound_names,compound_id,pathway,standards_match,spectra,adducts,hmdbacc,hmdbname,matchesHMDB,HMDBspecAnno
1401,203.05,12.05,0.00,7.89,1401,"['D-Tagatose; lyxo-Hexulose$1,7-Dimethylxanthi...",['E351'],"['Tryptophan metabolism', 'Caffeine metabolism...",NaN,{'frag_pos_mal': Spectrum from scan 2262 in /V...,"[(202.04510588796632, M+H[1+], ), (404.0902117...","[HMDB0062551, HMDB0000295, HMDB0014988, HMDB00...","[4-ethylphenylsulfate, Uridine 5'-diphosphate,...","[(HMDB0014988, 0.10091398134601763, Spectrum f...",[]
2664,468.31,3.97,0.00,-7.69,2664,['Timnodonyl carnitine'],['E385'],['Carnitine shuttle'],NaN,{'frag_pos_mal': Spectrum from scan 3642 in /V...,"[(467.30115930988427, M+H[1+], ), (934.6023186...","[HMDB0000998, HMDB0076294, HMDB0076299, HMDB00...","[dCTP, CL(a-13:0/i-19:0/18:2(9Z,11Z)/18:2(9Z,1...","[(HMDB0015325, 0.04792514380900029, Spectrum f...",[]
613,147.08,11.91,0.00,-7.21,613,"['5,6-Dihydrothymine; Dihydrothymine; 5,6-Dihy...","['E171', 'E237']","['Pyrimidine metabolism', 'Nitrogen metabolism...",NaN,{'frag_pos_mal': Spectrum from scan 1206 in /V...,"[(146.06913101597954, M+H[1+], ), (292.1382620...","[HMDB0000182, HMDB0000208, HMDB0000422, HMDB00...","[L-Lysine, Oxoglutaric acid, 2-Methylglutaric ...","[(HMDB0000641, 0.47978524104955117, Spectrum f...","[((146.1445, C5H10N2O3, L-Glutamine), 0.479785..."
324,126.02,12.21,0.00,-7.01,324,['Taurine; 2-Aminoethanesulfonic acid; Aminoet...,['E288'],"['Methionine and cysteine metabolism', 'Bile a...",NaN,{'frag_pos_mal': Spectrum from scan 699 in /Vo...,"[(125.0147042204216, M+H[1+], ), (250.02940844...","[HMDB0000251, HMDB0000898, HMDB0001861, HMDB00...","[Taurine, 1-Methylhistamine, 3-Methylhistamine...","[(HMDB0000479, 0.015941508107824173, Spectrum ...",[]
3108,904.59,3.00,0.00,-6.96,3108,NaN,row19,NaN,NaN,{'gnps2': Spectrum from scan 2029 in /Volumes/...,"[(903.5830648524384, M+H[1+], ), (1807.1661297...","[HMDB0005385, HMDB0005426, HMDB0005461, HMDB00...","[TG(16:0/18:1(9Z)/20:4(5Z,8Z,11Z,14Z)), TG(16:...","[(0, 0, 0)]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,158.12,8.66,0.92,0.12,777,NaN,row589,NaN,NaN,{'frag_pos_vl': Spectrum from scan 2096 in /Vo...,"[(157.11036220787932, M+H[1+], ), (314.2207244...","[HMDB0000459, HMDB0014447, HMDB0014755, HMDB00...","[3-Methylcrotonylglycine, Tranexamic Acid, Par...","[(HMDB0000517, 0.25613996616596296, Spectrum f...",[]
1822,239.14,4.38,0.94,-0.10,1822,NaN,row594,NaN,NaN,{'frag_pos_mal': Spectrum from scan 2794 in /V...,"[(238.13169486196526, M+H[1+], ), (476.2633897...","[HMDB0014562, HMDB0014836, HMDB0015084, HMDB00...","[Secobarbital, Nitrofurantoin, Felbamate, Biop...","[(HMDB0001964, 0.1534860731591503, Spectrum fr...",[]
636,148.10,6.83,0.96,-0.07,636,['Pantothenate; Pantothenic acid; (R)-Pantothe...,"['E23', 'E165']",['Vitamin B5 - CoA biosynthesis from pantothen...,NaN,{'frag_pos_mal': Spectrum from scan 1249 in /V...,"[(147.0895086919544, M+H[1+], ), (294.17901738...","[HMDB0000148, HMDB0003011, HMDB0003339, HMDB00...","[L-Glutamic acid, O-Acetylserine, D-Glutamic a...","[(HMDB0000017, 0.2308221620676222, Spectrum fr...",[]
805,159.11,10.41,0.98,-0.04,805,NaN,row599,NaN,NaN,{'frag_pos_vl': Spectrum from scan 2164 in /Vo...,"[(158.10559349863266, M+H[1+], ), (316.2111869...","[HMDB0000462, HMDB0000635, HMDB0000847, HMDB00...","[Allantoin, Succinylacetone, Pelargonic acid, ...","[(HMDB0000197, 0.17020043888508127, Spectrum f...",[]


In [165]:
for i in range(len(cids)):
    if cids[i] in ann.index:
        print(i,cids[i],ann.loc[cids[i]]['standards_match'])
        
        print(ann.loc[cids[i]]['matchesHMDB'])
        print(ann.loc[cids[i]]['HMDBspecAnno'])
        print(ann.loc[cids[i]]['compound_names'])
        
        print('==============================================================================')

0 1993 sn-glycero-3-Phosphocholine
[('HMDB0015576', 0.1310692376203601, <mnet.Spectrum object at 0x124a1e790>), ('HMDB0014695', 0.14199269357379657, <mnet.Spectrum object at 0x124a1e7c0>)]
[]
nan
1 952 nan
[('HMDB0000670', 0.31654856105313567, <mnet.Spectrum object at 0x124a1eaf0>)]
[]
nan
2 366 5-Oxoproline
[('HMDB0000897', 0.08628637561716006, <mnet.Spectrum object at 0x124a1ec40>), ('HMDB0000070', 0.36276145491431677, <mnet.Spectrum object at 0x124a1ec70>)]
[(('129.157', 'C6H11NO2', 'Pipecolic acid'), 0.36276145491431677)]
['1-Pyrroline-4-hydroxy-2-carboxylate$5-Oxo-L-proline$L-1-Pyrroline-3-hydroxy-5-carboxylate; 3-Hydroxy-L-1-pyrroline-5-carboxylate; (3R,5S)-1-Pyrroline-3-hydroxy-5-carboxylate$5-Oxoproline; Pyroglutamic acid; 5-Pyrrolidone-2-carboxylic acid; Pyroglutamate; 5-Oxo-L-proline; L-Pyroglutamic acid; L-5-Pyrrolidone-2-carboxylic acid', 'L-4-Hydroxyglutamate semialdehyde$D-Glutamate; D-Glutamic acid; D-Glutaminic acid; D-2-Aminoglutaric acid$L-Glutamate; L-Glutamic acid; 

In [125]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(with_mod)

          logFC        t  P.Value  adj.P.Val   row m/z   row rt     mal      vl     zik
3022  -11.33130 -8.69038  0.00000    0.00000 833.58745  3.13144    None    None 0.98242
1517  -10.24873 -8.57249  0.00000    0.00000 213.09409 11.95398 0.96493 0.00000 0.61075
2947   -2.08999 -8.22249  0.00000    0.00000 788.61208  3.18832 0.76168 0.00001 0.35794
1401    2.54919  7.88909  0.00000    0.00000 203.05263 13.54806    None 0.14643 0.82484
101    -2.08820 -7.76761  0.00000    0.00000 101.07178 12.98222    None    None    None
2664   -1.90723 -7.68673  0.00000    0.00000 469.31196  3.97423 0.13870 0.01179 0.80425
2987   -1.86304 -7.61452  0.00000    0.00000 810.60001  3.17127 0.90425 0.00000 0.94713
2175    1.87656  7.58335  0.00000    0.00000 280.26343 22.69388    None    None 0.97127
2985   -2.02375 -7.30058  0.00000    0.00000 809.58754  3.18517 0.95649 0.00000 0.85724
2912   -1.59756 -7.22493  0.00000    0.00000 768.55257  3.12411 0.98420 0.00000 0.55699
2549   -0.94566 -7.22200  0.0000

In [89]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(with_mod)

           logFC         t       P.Value     adj.P.Val     row m/z     row rt   mal           vl
3022  -11.331295 -8.690375  5.255238e-13  2.670170e-10  833.587449   3.131435  None         None
1517  -10.248729 -8.572486  8.841622e-13  2.670170e-10  213.094092  11.953978  None         None
2947   -2.089988 -8.222488  4.142949e-12  8.341137e-10  788.612085   3.188320  None  5.18657e-06
1401    2.549193  7.889092  1.800742e-11  2.719120e-09  203.052629  13.548055  None         None
101    -2.088202 -7.767609  3.073015e-11  3.712202e-09  101.071779  12.982221  None         None
...          ...       ...           ...           ...         ...        ...   ...          ...
10779  -0.006983 -0.032195  9.744013e-01  9.808212e-01  170.153955   9.201242  None         None
2057    0.007572  0.030247  9.759495e-01  9.808212e-01  268.176428   9.853239  None         None
2476    0.004093  0.015505  9.876699e-01  9.909512e-01  344.227831   4.196603  None         None
1393   -0.001946 -0.012650  9.

In [81]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [28]:
nomod = {}
mod = {}
for _,row in without_mod.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    nomod[round(mz,2)] = idc
for _,row in with_mod.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    mod[round(mz,2)] = idc

In [37]:
def intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 
      
inters  = intersection(mod.keys(),  nomod.keys())

In [55]:
found = []
notfound = []
for i in mod:
    if i in list(inters):
        found = mod[i]
    else:
        notfound = mod[i]


In [ ]:
mod_sig = with_mod[with_mod['adj p-val'] < 0.05]
nomod_sig = without_mod[without_mod['adj p-val'] < 0.05]


mal_sig = mal[mal['adj-p'] < 0.05]


vl_sig = vl[vl['adj-p'] < 0.05]

zik_sig = zik[zik['adj-p'] < 0.05]


mal_sig_new = mal_new[mal_new['adj-p'] < 0.05]


vl_sig_new = vl_new[vl_new['adj-p'] < 0.05]


nomod = {}
mod = {}

mod_rt = []
nomod_rt = []

mod_mz = []
nomod_mz = []

for _,row in nomod_sig.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    nomod[round(mz,2)] = idc
    nomod_rt.append(rt)
    nomod_mz.append(mz)
    
for _,row in mod_sig.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    mod[round(mz,2)] = idc
    mod_rt.append(rt)
    mod_mz.append(mz)

def intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 
      
inters  = intersection(mod.keys(),  nomod.keys())


found = []
notfound = []
for i in mod:
    if i in list(inters):
        found.append(mod[i])
    else:
        notfound.append(mod[i])

        
names_notfound = []
index = 0
for i in notfound:
    name  = str(data.loc[i][5]).strip()
    if name != "NaN":
        if name != "nan":
            index+=1
            print("========",index,"=======")
            print(name)
            names_notfound.append(name)
            print("")   
            
            
names_found = []
index = 0
for i in found:
    name  = str(data.loc[i][5]).strip()
    if name != "NaN":
        if name != "nan":
            index+=1
            print("========",index,"=======")
            print(name)
            names_found.append(name)
            print("")
            
for i in (list(intersection(names_notfound, names_found))):
    print("=============")
    print(i)
    
for i in names_notfound:
    if i not in (list(intersection(names_notfound, names_found))):
        print("=============")
        print(i)

### Check intersection metabolites by m/z

In [ ]:
inters = 0
for _,row in nomod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,rowc in mod_sig.iterrows():
        mzc = rowc['row m/z']
        rtc = rowc['row retention time']
        if (mz < (mzc+0.001)  or  mz > (mzc-0.001)):
            inters+=1
            break
            #print(row)

In [ ]:
print("Mal sig: {}, VL sig: {}, Zik sig: {}".format(len(mal_sig), len(vl_sig), len(zik_sig)))

In [ ]:
c=0
z=0
for _,row in nomod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,row in vl_sig.iterrows():
        mz_vl = row['row m/z']
        rt_vl = row['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 300) and rt_vl < (rt +300):
                z+=1
print(c,z)

### Check retention times

In [ ]:
plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.xlabel('RT')
plt.ylabel('Peaks (#)')


unpreprocessed data: Most of the peaks which align are found in the interval [3,7] and very few in [8,14]



preprocessed data: More peaks align in the interval [8,14]

So the modelling mainly influences the metabolites eluted after 8 min.

In [ ]:
c=0
z=0
for _,row in vl_sig_new.iterrows():
    mz = row['row m/z']
    mz = np.round(mz,3)
    rt = row['row retention time']
    for _,rowl in zik_sig.iterrows():
        mz_vl = rowl['row m/z']
        
        rt_vl = rowl['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 30/60) and rt_vl < (rt +30/60):
                z+=1
                
print(c,z)


c=0
z=0
for _,row in mal_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,row in zik_sig.iterrows():
        mz_vl = row['row m/z']
        rt_vl = row['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 300) and rt_vl < (rt +300):
                z+=1
print(c,z)

cv=0
zv=0
for _,row in mod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,rowl in nomod_sig.iterrows():
        mz_vl = rowl['row m/z']
        rt_vl = rowl['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            cv+=1
            if rt_vl > (rt - 300/60) and rt_vl < (rt +300/60):
                zv+=1
                print(mz, rt)
                print(mz_vl, rt_vl)
            print('-------')
print(cv,zv)

plt.hist(mod_mz,)
plt.hist(nomod_mz, alpha = 0.5)

plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.hist(mod_mz, color = 'pink')
plt.hist(nomod_mz, alpha = 0.5)
plt.hist(mod_sig['row m/z'], alpha = 0.4)
plt.hist(nomod_sig['row m/z'], alpha = 0.3)


plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=100)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=100)

plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

sns.distplot(mod_sig['row m/z'])
sns.distplot(nomod_sig['row m/z'])

sns.distplot(mod_sig['row retention time'])
sns.distplot(nomod_sig['row retention time'])

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row retention time'], bins=100)
plt.hist(nomod_sig['row retention time'], alpha = 0.5, bins=100)

plt.xlabel('RT')
plt.ylabel('Peaks (#)')


plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(vl_sig['row m/z'], alpha = 0.5, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(zik_sig['row m/z'], alpha = 0.8, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(mal_sig['row m/z'], alpha = 0.8, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')


plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.hist(zik['row retention time'], alpha = 0.5)

plt.xlabel('RT')
plt.ylabel('Peaks (#)')

plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.hist(mal['row retention time'], alpha = 0.5)
plt.xlabel('RT')
plt.ylabel('Peaks (#)')